In [91]:
#Dependencies 
import pandas as pd
import wget

In [92]:
# url of the raw csv dataset
urls = [
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
]

# download data using wget function
[wget.download(url) for url in urls]

['time_series_covid19_confirmed_global.csv',
 'time_series_covid19_deaths_global.csv',
 'time_series_covid19_recovered_global.csv']

In [93]:
# load the data
confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('time_series_covid19_recovered_global.csv')

# checking df
confirmed_df
deaths_df
recovered_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,23924,24550,24602,24793,25180,25198,25358,25389,25471,25509
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,2463,2523,2608,2637,2682,2745,2789,2830,2883,2952
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,16983,17369,17369,18076,18088,18837,19233,19592,20082,20537
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,803,803,803,803,803,803,803,804,806,807
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,221,236,241,242,242,242,266,301,395,437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,NaN,Sao Tome and Principe,0.186400,6.613100,0,0,0,0,0,0,...,601,604,610,662,696,734,759,766,771,778
249,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,751,762,779,780,797,833,840,849,856,862
250,NaN,Comoros,-11.645500,43.333300,0,0,0,0,0,0,...,319,324,324,324,328,328,328,330,330,330
251,NaN,Tajikistan,38.861000,71.276100,0,0,0,0,0,0,...,5741,5793,5851,5906,5970,6028,6065,6103,6151,6193


In [94]:
# extracting all of the dates columns
dates = confirmed_df.columns[4:]

# merging all of the confitmed, deaths, and recovered cases using melt function
# at this step we are using province, country, lat, and long as variables for merging
confirmed_df_long = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
deaths_df_long = deaths_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
recovered_df_long = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)

# checking the dataframe
confirmed_df_long
deaths_df_long
recovered_df_long

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
48571,NaN,Sao Tome and Principe,0.186400,6.613100,7/31/20,778
48572,NaN,Yemen,15.552727,48.516388,7/31/20,862
48573,NaN,Comoros,-11.645500,43.333300,7/31/20,330
48574,NaN,Tajikistan,38.861000,71.276100,7/31/20,6193


In [95]:
# removing Canada data because it was noted in the datasource about the data mismatched
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']

# merging two variables at a time
full_table = confirmed_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

# merging again with recovered data
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

# create tracker for active case (active - confirmed - death - recovered)
full_table['Active'] = full_table["Confirmed"] - full_table['Deaths'] - full_table['Recovered']

# checking the df
full_table


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0,0,0.0,0.0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0.0,0.0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0.0,0.0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0.0,0.0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
51067,NaN,Sao Tome and Principe,0.186400,6.613100,7/31/20,871,15,778.0,78.0
51068,NaN,Yemen,15.552727,48.516388,7/31/20,1728,493,862.0,373.0
51069,NaN,Comoros,-11.645500,43.333300,7/31/20,378,7,330.0,41.0
51070,NaN,Tajikistan,38.861000,71.276100,7/31/20,7409,60,6193.0,1156.0


In [96]:
# convert date to date time
full_table["Date"] = pd.to_datetime(full_table['Date'])
full_table

# replacing N/A
full_table['Recovered'] = full_table['Recovered'].fillna(0)


In [97]:
# grouping data by setting index and summing
full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [98]:
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
36091,2020-07-31,West Bank and Gaza,11837,82,5077.0,6678.0
36092,2020-07-31,Western Sahara,10,1,8.0,1.0
36093,2020-07-31,Yemen,1728,493,862.0,373.0
36094,2020-07-31,Zambia,5963,151,3803.0,2009.0


In [99]:
full_grouped.to_csv('COVID-19-time-series-clean-complete.csv')